# Center Protein in Box

In [1]:
import numpy as np
import MDAnalysis
import pandas as pd
from tqdm.notebook import tqdm_notebook
import time

In [26]:
## STRUCTURE NEEDS TO BE A TPR
structure = 'prodreplic2m3.tpr'
trajectory = 'prodreplic2m3.xtc'
out = 'translated.xtc'

In [27]:
u = MDAnalysis.Universe(structure, trajectory)
allatoms=u.select_atoms("all")
pep=u.residues[:31]

In [31]:
universe = u.select_atoms("all")

print(pep.center_of_mass())
with tqdm_notebook(total=len(u.trajectory[:5])) as pbar:
    with MDAnalysis.Writer(out, universe.n_atoms) as w:
        for ts in u.trajectory[:5]:

            universe.unwrap(compound="residues")
            dim = ts.triclinic_dimensions
            box_center = np.sum(dim, axis=0) / 2
            
            u.atoms.translate([(box_center[0]-pep.center_of_mass()[0]),(box_center[1]-pep.center_of_mass()[1]), 0])
            universe.wrap(compound="residues")
            w.write(universe)

            pbar.update(1)

[49.80333047 60.3318956  73.8530112 ]


  0%|          | 0/5 [00:00<?, ?it/s]

In [38]:
def center_peptide(universe, peptide, output, skip=1):
    selection = u.select_atoms("all")

    with tqdm_notebook(total=len(universe.trajectory[::skip])) as pbar:
        with MDAnalysis.Writer(output, selection.n_atoms) as w:
            for ts in universe.trajectory[::skip]:

                selection.unwrap(compound="residues")
                dim = ts.triclinic_dimensions
                box_center = np.sum(dim, axis=0) / 2

                universe.atoms.translate([(box_center[0]-pep.center_of_mass()[0]),(box_center[1]-pep.center_of_mass()[1]), 0])
                selection.wrap(compound="residues")
                w.write(selection)

                pbar.update(1)

In [39]:
center_peptide(u, u.residues[:31], "translate_peptide1.xtc", skip=500)

  0%|          | 0/21 [00:00<?, ?it/s]

ValueError: AtomGroup was not contiguous from bonds, process failed

In [ ]:
center_peptide(u, u.residues[31:62], "translate_peptide2.xtc", skip=100)